Acknowledgement:
    1. The structure of the code is inspired by depmixS4: An R Package for Hidden Markov Models: https://cran.r-project.org/web/packages/depmixS4/vignettes/depmixS4.pdf
    2. Some of the linear model codes are adapted from sklearn: http://scikit-learn.org/stable/ and statsmodel: http://statsmodels.sourceforge.net/. Some modifications have been made to these codes to realize more functionalities.

Problems with existing packages
    1. Some of sklearn and statsmodels does not support the implementation of sample weights
    2. Some of sklearn and statsmodels does not support l1, l2 or elasticnet regularizations
    3. Sklearn packages does not support estimation of standard deviation of coefficients
    4. The likelihood function of weighted linear models is not the same as the ones we need to use in IO-HMM
    5. In the R package aformentioned, they do not support the provision of multiple sequences.

Modifications to above packages:
    1. Implemented supervised models that supports sample weights
    2. Supports the estimation of standard deviations of coefficients
    3. Supports multiple regularizations (l1, l2, elastic net) in most of the supervised models. (However,  if regularization is applied, no standard deviation of the coefficients will be estimated)
    4. Supports estimation over multiple sequences (multiple dataframes)
    5. HMM forward-backward code was implemented at the log scale so that it is more robust to long sequences.
    6. Supports generalized linear models with different link functions, just as statsmodel.

# Example use of UnSupervised_IOHMM

In [1]:
%load_ext autoreload
%autoreload 2

from __future__ import  division

from copy import deepcopy
import sys
import warnings


import numpy as np
import pandas as pd
from scipy.misc import logsumexp


from IOHMM import UnSupervisedIOHMM, UnSupervisedIOHMMMapReduce
from IOHMM import LM, MNLP, MNLD


warnings.simplefilter("ignore")

## Speed data - example 1

In [2]:
speed = pd.read_csv('../data/speed.csv')
speed.head()

,Unnamed: 0,rt,corr,Pacc,prev
0,1,6.456770,cor,0.0,inc
1,2,5.602119,cor,0.0,cor
2,3,6.253829,inc,0.0,cor
3,4,5.451038,inc,0.0,inc
4,5,5.872118,inc,0.0,inc


## Setting up the model

In [3]:
SHMM = UnSupervisedIOHMM(num_states=2, max_EM_iter=1000, EM_tol=1e-2)
SHMM.setModels(model_emissions = [LM()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[]])
SHMM.setOutputs([['rt']])
SHMM.setData([speed])

In [4]:
SHMM.train()

## See the coefficients

In [5]:
print np.exp(SHMM.model_transition[0].coef - logsumexp(SHMM.model_transition[0].coef))
print np.exp(SHMM.model_transition[1].coef - logsumexp(SHMM.model_transition[1].coef))

[[ 0.80043352  0.19956648]]
[[ 0.08965284  0.91034716]]


In [6]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[1][0].coef

[ 5.50216013]
[ 6.38011574]


In [7]:
print np.sqrt(SHMM.model_emissions[0][0].dispersion)
print np.sqrt(SHMM.model_emissions[1][0].dispersion)

0.182711715962
0.247049173969


## Speed data - example 2

In [8]:
SHMM = UnSupervisedIOHMM(num_states=2, max_EM_iter=1000, EM_tol=1e-2)
SHMM.setModels(model_emissions = [LM(est_sd = True), MNLD(est_sd=True)], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[],['Pacc']])
SHMM.setOutputs([['rt'],['corr']])
SHMM.setData([speed])

In [9]:
SHMM.train()

In [10]:
print np.exp(SHMM.model_transition[0].coef - logsumexp(SHMM.model_transition[0].coef))
print np.exp(SHMM.model_transition[1].coef - logsumexp(SHMM.model_transition[1].coef))

[[ 0.90699325  0.09300675]]
[[ 0.19975045  0.80024955]]


In [11]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[0][1].coef
print SHMM.model_emissions[1][0].coef
print SHMM.model_emissions[1][1].coef

[ 6.38141004]
[[ 0.         -0.99623318]
 [ 0.         -2.42428868]]
[ 5.50367124]
[[ 0.         -0.2204467 ]
 [ 0.          0.62768677]]


In [12]:
print SHMM.model_emissions[0][0].sd
print SHMM.model_emissions[0][1].sd
print SHMM.model_emissions[1][0].sd
print SHMM.model_emissions[1][1].sd

[ 0.0150907]
[[ 0.          0.37302218]
 [ 0.          0.79613534]]
[ 0.01363804]
[[ 0.          0.15889117]
 [ 0.          0.73583209]]


## MapReduce Version

In [14]:
sc.stop()
sc = SparkContext(appName="")

In [15]:
speed = pd.read_csv('../data/speed.csv', index_col=0)
indexes = [(1,1), (2,1)]
RDD = sc.parallelize(indexes)
dfs = RDD.mapValues(lambda v: speed)

In [16]:
SHMM = UnSupervisedIOHMMMapReduce(num_states=2, max_EM_iter=100, EM_tol=1e-4)
SHMM.setModels(model_emissions = [LM()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[]])
SHMM.setOutputs([['rt']])
SHMM.setData(dfs)

In [17]:
SHMM.train()
print 'done'

done


In [18]:
sc.stop()

In [20]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[1][0].coef

[ 5.51005132]
[ 6.38494108]


In [21]:
print np.exp(SHMM.model_transition[0].coef - logsumexp(SHMM.model_transition[0].coef))
print np.exp(SHMM.model_transition[1].coef - logsumexp(SHMM.model_transition[1].coef))

[[ 0.88162185  0.11837815]]
[[ 0.08845407  0.91154593]]
